In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm.notebook import tqdm

from typing import Literal

In [ ]:
MAX_LAYERS = 42
RAW_SIZE = sum([p.size for p in packages])
COMPRESSION_RATIO = 12 / 4.6  # This is for bazzite

# Some layers will naturally be bigger, due to containing
# large packages. Therefore, the first prefill step will
# only fill layers up to X% before stopping.
# 0.9 is the saturation point during testing
FILL_PERCENT = 0.7
# The algorithm might start adding packages to layers that
# are already large, block that.
# FIXME: this being too low might cause an error
MAX_LAYER_PERCENT = 1.3

# Experiment: attempt to create the layers without considering the package added
# Therefore, only frequency changes will affect which package is added.
IGNORE_SIZE = False

LAYER_SIZE = RAW_SIZE / MAX_LAYERS
PREFILL_LAYER_SIZE = FILL_PERCENT * LAYER_SIZE
MAX_LAYER_SIZE = MAX_LAYER_PERCENT * LAYER_SIZE
print(f'Raw size: {RAW_SIZE / 1e9:.3f} GB')
print(f"Prefill layer size: {PREFILL_LAYER_SIZE / 1e9:.3f} GB")
print(f"Max layer size: {MAX_LAYER_SIZE / 1e9:.3f} GB")

In [ ]:
todo = dict(prefill_todo)
pbar = tqdm(total=len(todo), desc="Final layer fill")
layers = [l.copy() for l in prefill_layers]

layer_size = [sum([p.size for p in l]) for l in layers]
layer_upd = [np.zeros(SEGMENTS, dtype=np.bool) for _ in range(len(layers))]
for l in layers:
    for p in l:
        layer_upd[layers.index(l)] |= p_upd[p.index]
layer_bw = [float(np.sum(layer_upd[i]) * layer_size[i]) for i in range(len(layers))]

while todo:
    # There are still some leftover packages.
    # Since we did a heuristic to prefill the layers
    # we left out some space.
    #
    # Now we go back and do a computationally expensive step
    # and insert the packages in the layers that will cause
    # the least bandwidth increase.
    b_layer = None
    b_pkg = None
    b_upd = None
    b_bw = 1e24
    b_bw_total = 0
    for i in range(len(layers)):
        l_size = layer_size[i]
        if l_size > MAX_LAYER_SIZE:
            continue
        for p in todo:
            upd = layer_upd[i] | p_upd[p.index]
            new_size = l_size + p.size
            bw_total = np.sum(upd) * (new_size if IGNORE_SIZE else l_size)
            bw = bw_total - layer_bw[i]
            if bw < b_bw:
                b_bw = bw
                b_layer = i
                b_pkg = p
                b_upd = upd
                b_bw_total = bw_total

    assert (
        b_layer is not None and b_pkg is not None and b_upd is not None
    ), "No package selected. How did we get here?"

    layers[b_layer].append(b_pkg)
    layer_upd[b_layer] = b_upd
    layer_size[b_layer] += b_pkg.size
    layer_bw[b_layer] = float(b_bw_total)
    todo.pop(b_pkg)
    pbar.update(1)

pbar.close()

# Logging
print(f"Final layer fill:")
for i, (l, pl) in enumerate(zip(layers, prefill_layers)):
    log = f"Layer {i+1:2d}: {sum([p.size for p in l]) / 1e6:3.0f} MB"
    log += f"(c {sum([p.size for p in l]) / 1e6 / COMPRESSION_RATIO:3.0f}) with {len(l):3d}"
    if len(pl) != len(l):
        log += f" packages (prev {sum([p.size for p in pl]) / 1e6:3.0f} MB with {len(pl):3d})."
    else:
        log += " packages."
    print(log)

print()

total_bw = 0
for i, l in enumerate(layers):
    total_bw += np.sum(layer_upd[i]) * sum([p.size for p in l])

print(f"Total per update (uncompressed): {total_bw / (SEGMENTS * 1e9):.3f} GB.")
print(f"Total per update (compressed): {total_bw / (SEGMENTS * 1e9) / COMPRESSION_RATIO:.3f} GB.")
print(
    f"Layers changed per update: {np.sum([np.sum(u) for u in layer_upd]) / SEGMENTS:.1f}."
)

In [ ]:
print(f"Packages in layers:")
for i, l in sorted(enumerate(layers), key=lambda x: -float(np.sum(layer_upd[x[0]]))):
    print(
        f"{i+1:3d}: (pkg: {len(l):3d}, freq: {np.sum(layer_upd[i]):3d}, mb: {sum([p.size for p in l]) / 1e6 / COMPRESSION_RATIO:3.0f})",
        [p.nevra for p in l],
    )